<a href="https://colab.research.google.com/github/HammadN98/previsao_de_vendas/blob/main/previsao_das_vendas_lojas_Rossmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Importacoes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
store = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/store.csv")
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/train.csv")
train.head()

<ipython-input-3-24519896a09f>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/train.csv")


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
test = pd.read_csv("https://raw.githubusercontent.com/HammadN98/previsao_de_vendas/refs/heads/main/rossmann-store-sales/test.csv")
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


juntar os datasets para garantir ter todas as variaveis disponiveis

In [5]:
#Dimensoes pre-merge
store.shape, train.shape, test.shape

((1115, 10), (1017209, 9), (41088, 8))

In [6]:
train = train.merge(store, on='Store', how='left')
train.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [7]:
test = test.merge(store, on='Store', how='left')
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN


In [8]:
#Dimensoes pOS-merge
train.shape, test.shape


((1017209, 18), (41088, 17))

## 1. Analise Exploratoria

### 1.1 Valores ausentes

In [9]:
#Valores ausentes por coluna no train
train.isnull().sum()

,0
Store,0
DayOfWeek,0
Date,0
Sales,0
Customers,0
Open,0
Promo,0
StateHoliday,0
SchoolHoliday,0
StoreType,0


In [10]:
#Valores ausentes por coluna no test
test.isnull().sum()

,0
Id,0
Store,0
DayOfWeek,0
Date,0
Open,11
Promo,0
StateHoliday,0
SchoolHoliday,0
StoreType,0
Assortment,0


A coluna usada como base para o merge ficou com 0 NaN

In [11]:
#PErcentual de valores ausentes por coluna
print(train.isnull().mean() * 100)
print(test.isnull().mean() * 100)


Store                         0.000000
DayOfWeek                     0.000000
Date                          0.000000
Sales                         0.000000
Customers                     0.000000
Open                          0.000000
Promo                         0.000000
StateHoliday                  0.000000
SchoolHoliday                 0.000000
StoreType                     0.000000
Assortment                    0.000000
CompetitionDistance           0.259730
CompetitionOpenSinceMonth    31.787764
CompetitionOpenSinceYear     31.787764
Promo2                        0.000000
Promo2SinceWeek              49.943620
Promo2SinceYear              49.943620
PromoInterval                49.943620
dtype: float64
Id                            0.000000
Store                         0.000000
DayOfWeek                     0.000000
Date                          0.000000
Open                          0.026772
Promo                         0.000000
StateHoliday                  0.000000
SchoolHoli

OS valores faltantes serao substituidos por:

CompetitionDistance -> Mediana

Open -> 0 / A loja devia estar fechada

CompetitionOpenSinceMonth, CompetitionOpenSinceYear -> 0 / indica que nao tem competidores proximos

Promo2SinceWeek, Promo2SinceYear, PromoInterval -> 0 / A loja nao participas das promocoes

In [12]:
train['CompetitionDistance']

,CompetitionDistance
0,1270.0
1,570.0
2,14130.0
3,620.0
4,29910.0
...,...
1017204,1900.0
1017205,1880.0
1017206,9260.0
1017207,870.0


In [13]:
median_value_train = train['CompetitionDistance'].median()
train['CompetitionDistance'] = train['CompetitionDistance'].fillna(median_value_train)
median_value_test = test['CompetitionDistance'].median()
test['CompetitionDistance'] = test['CompetitionDistance'].fillna(median_value_test)

train['CompetitionOpenSinceMonth'] = train['CompetitionOpenSinceMonth'].fillna(0)
test['CompetitionOpenSinceMonth'] = test['CompetitionOpenSinceMonth'].fillna(0)

train['CompetitionOpenSinceYear'] = train['CompetitionOpenSinceYear'].fillna(0)
test['CompetitionOpenSinceYear'] = test['CompetitionOpenSinceYear'].fillna(0)

train['Promo2SinceWeek'] = train['Promo2SinceWeek'].fillna(0)
test['Promo2SinceWeek'] = test['Promo2SinceWeek'].fillna(0)

train['Promo2SinceYear'] = train['Promo2SinceYear'].fillna(0)
test['Promo2SinceYear'] = test['Promo2SinceYear'].fillna(0)

train['PromoInterval'] = train['Promo2SinceYear'].fillna('None')
test['PromoInterval'] = test['PromoInterval'].fillna('None')

test['Open'] = test['Open'].fillna(0)

In [14]:
print("Valores ausentes após o tratamento (train):")
print(train.isnull().sum())

Valores ausentes após o tratamento (train):
Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64


In [15]:
print("\nValores ausentes após o tratamento (test):")
print(test.isnull().sum())


Valores ausentes após o tratamento (test):
Id                           0
Store                        0
DayOfWeek                    0
Date                         0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64


Dados faltantes tratados

## 2. Analise Descritiva

In [16]:
train.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01,5.422034e+03,4.926878e+00,1.370173e+03,5.005638e-01,1.164767e+01,1.007011e+03,1.007011e+03
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01,7.706913e+03,4.283543e+00,9.353634e+02,4.999999e-01,1.532393e+01,1.005877e+03,1.005877e+03
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00,7.100000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00,2.330000e+03,4.000000e+00,2.006000e+03,1.000000e+00,1.000000e+00,2.009000e+03,2.009000e+03
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00,6.880000e+03,9.000000e+00,2.011000e+03,1.000000e+00,2.200000e+01,2.012000e+03,2.012000e+03
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00,7.586000e+04,1.200000e+01,2.015000e+03,1.000000e+00,5.000000e+01,2.015000e+03,2.015000e+03


#### Insights:

Store: Vai de 1 a 1115, indicando 1115 loja unicas.

DayOfWeek: VAi de 1 (Segunda) a 7 (Domingo).

Sales: VEndas minimas = 0, possivelvente a loja estava fechada, a media fica em 5773.82, com um desvio padrao de 3849.93 o que indica uma altissima variacao. E tando o maximo em 41,551.

Custumers: MIN -> 0(Loja fechada possivelmente). Media -> 633. Dp -> 464, indicando alta variacao.  MAX -> 7388.

Promo: A media indica que 38% dos dados tem promocoes em atividade.

Promo2: A media indica que +50% nao participa das promocoes 2

CompetitionDistance: MIN -> 2km indica proximidade de concorrentes, Media -> 5,4km. MAX -> 75km, lojas mais isoladas.

CompetitionOpenSinceYear -> OS valores 0, puxaram a media para 1370. Indicando que a escolha da codificacao nao foi efetiva. Retomar a esse ponto.

Open: 83% da lojas abertas

Holiday(School e State): Grande maioria 0, o que era esperado.






In [20]:
for categorica in ['StoreType', 'Assortment', 'StateHoliday']:
    print(f"{train[categorica].value_counts()}\n")


StoreType
a    551627
d    312912
c    136840
b     15830
Name: count, dtype: int64

Assortment
a    537445
c    471470
b      8294
Name: count, dtype: int64

StateHoliday
0    855087
0    131072
a     20260
b      6690
c      4100
Name: count, dtype: int64



In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1017209 non-null  float64
 12  CompetitionOpenSinceMonth  1017209 non-null  float64
 13  CompetitionO